In [1]:
import numpy as np
import socket
import time
from Go import GameState as Go, check_possible_moves, is_game_finished

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
games = ["A4x4", "A5x5", "A6x6", "G7x7", "G9x9"]
game = games[3]

In [3]:
def is_valid_move_go(game,i,j):    # implementing the logic to check if the move is valid
    return (i,j) in check_possible_moves(game)

In [4]:
def start_server_go(host='localhost', port=12345):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(2)

    print("Waiting for two agents to connect...")
    agent1, addr1 = server_socket.accept()
    print("Agent 1 connected from", addr1)
    bs=b'AG1 '+game.encode()
    agent1.sendall(bs)

    agent2, addr2 = server_socket.accept()
    print("Agent 2 connected from", addr2)
    bs=b'AG2 '+game.encode()
    agent2.sendall(bs)
       
    n = int(game[1])
    initial_board = np.zeros((n, n),dtype=int)     # initializing an empty board of size (n x n)
    GameState = Go(initial_board)    # initializing the game

    agents = [agent1, agent2]
    current_agent = 0

    jog=0
    
    while True:
        try:
            data = agents[current_agent].recv(1024).decode()
            if not data:
                break

            if data == "PASS":
                agents[current_agent].sendall(b'VALID')
                agents[1-current_agent].sendall(data.encode())
                GameState.pass_turn()
            else:
                # processing the move (example: "MOVE X,Y")
                i = int(data[5])
                j = int(data[7])
                print(current_agent, " -> ",data)
                jog = jog+1
                
                # checking if the move is valid and, if so, executing it
                if is_valid_move_go(GameState,i,j):
                    agents[current_agent].sendall(b'VALID')
                    agents[1-current_agent].sendall(data.encode())
                    GameState = GameState.move(i,j)
                else:
                    agents[current_agent].sendall(b'INVALID')
                    continue
                
            # checking if the game is over
            if is_game_finished(GameState):
                GameState.end_game()
                winner = GameState.winner
                p1_score = GameState.scores[1]
                p2_score = GameState.scores[-1]
                data = "END " + str(winner) + " " + str(p1_score) + " " + str(p2_score)
                agents[current_agent].sendall(data.encode())
                agents[1-current_agent].sendall(data.encode())
                break
                
            # Switch to the other agent
            current_agent = 1-current_agent
            time.sleep(1)

        except Exception as e:
            print("Error:", e)
            break

    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    agent1.close()
    agent2.close()
    server_socket.close()


In [5]:
if __name__ == "__main__":
    if game[0]=='G':
        start_server_go()

Waiting for two agents to connect...
Agent 1 connected from ('127.0.0.1', 53429)
Agent 2 connected from ('127.0.0.1', 53431)
0  ->  MOVE 8,4
0  ->  MOVE 6,8
0  ->  MOVE 5,8
0  ->  MOVE 7,3
0  ->  MOVE 6,6
1  ->  MOVE 6,7
1  ->  MOVE 8,4
1  ->  MOVE 6,6
1  ->  MOVE 7,4
1  ->  MOVE 4,8
1  ->  MOVE 4,7
1  ->  MOVE 2,5
0  ->  MOVE 0,6
1  ->  MOVE 8,3
1  ->  MOVE 4,3
0  ->  MOVE 5,2
1  ->  MOVE 4,7
1  ->  MOVE 7,9
1  ->  MOVE 9,8
1  ->  MOVE 0,2
0  ->  MOVE 4,1
1  ->  MOVE 2,9
1  ->  MOVE 3,6
0  ->  MOVE 7,0
0  ->  MOVE 4,4
1  ->  MOVE 5,8
1  ->  MOVE 3,1
0  ->  MOVE 4,6
1  ->  MOVE 5,0
0  ->  MOVE 5,4
1  ->  MOVE 3,7
1  ->  MOVE 7,5
1  ->  MOVE 9,8
1  ->  MOVE 3,8
1  ->  MOVE 7,0
1  ->  MOVE 4,5
0  ->  MOVE 9,3
0  ->  MOVE 3,6
0  ->  MOVE 5,5
1  ->  MOVE 6,9
1  ->  MOVE 5,2
1  ->  MOVE 6,1
0  ->  MOVE 9,2
0  ->  MOVE 8,0
0  ->  MOVE 5,4
0  ->  MOVE 5,7
0  ->  MOVE 0,2
0  ->  MOVE 5,7
0  ->  MOVE 8,3
0  ->  MOVE 5,5
0  ->  MOVE 3,2
1  ->  MOVE 4,2
0  ->  MOVE 8,2
0  ->  MOVE 8,1
0  ->  MOVE